In [ ]:
import openai
from openai import OpenAI
import os
import csv
import tiktoken
import pandas as pd

GPT_API_KEY = 'OPENAI_API_KEY'
os.environ['OPENAI_API_KEY'] = GPT_API_KEY
openai.api_key = GPT_API_KEY
client = OpenAI()

In [ ]:
data = pd.read_csv("Dailyhunt_Hindi.csv")
tokenizer = tiktoken.get_encoding('cl100k_base')

In [ ]:
def generate_summary(text):
    # Ensure the text is a string
    if not isinstance(text, str):
        return None

    # Encode the text to get tokens
    tokens = tokenizer.encode(text)
    token_count = len(tokens)

    # Token limit
    token_limit = 7500

    # Trim the text if the token count exceeds the limit
    if token_count > token_limit:
        text = text[:token_limit]

        # Recalculate tokens to ensure it's under the limit
        tokens = tokenizer.encode(text)
        while len(tokens) > token_limit:
            text = text[:-1]
            tokens = tokenizer.encode(text)

    # Call the model with the (possibly trimmed) text
    completion = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": "You are a human summarizing a news article. You have to summarize the text in the same language as the text. When anyone reads the summary, it should feel like it is written by a human and not a machine or AI."
            },
            {
                "role": "user",
                "content": f"Generate a summary using the contents. The summary should contain the main key points and facts of the article and should have minimal noise. Anybody reading the summary should be able to gain all the information of the original article and should feel like it is written by a human and not a machine. The generated summary should be in the same language as the original content.\n\nContents: {text}\n\n"
            }
        ]
    )

    # Retrieve the generated summary
    summary = completion.choices[0].message.content
    return summary

In [ ]:
# Open the output CSV file in append mode
with open('Hindi_Reference_Summary.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)

    # Write the header
    writer.writerow(data.columns.tolist() + ['Reference Summary'])

    # Iterate through the DataFrame rows with a simple counter
    total_rows = len(data)
    for index, row in enumerate(data.iterrows(), 1):
        # Generate summary
        summary = generate_summary(row[1]['Content'])

        # Prepare the row data
        row_data = row[1].tolist() + [summary]

        # Write the row to the CSV file
        writer.writerow(row_data)

        # Flush the file to ensure it's written immediately
        f.flush()

        # Print progress
        print(f"Processed article {index} of {total_rows}")

print("Summaries generated and saved to 'Hindi_Reference_Summary.csv'")

DataFrame saved to 'Bangla_Reference_Summary.csv'
